In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
filepath = '/content/drive/MyDrive/Web Data Analytics/group_project/'

### Pre processing

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import pandas as pd
df_0 = pd.read_csv(filepath+"Anagrafica_prodotto.csv")

In [ ]:
df_0.drop_duplicates(subset=['description', 'title'], keep="first")

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [ ]:
df=df_0.copy()
df.drop(columns=['Unnamed: 0','designation','province','region_1','region_2','taster_name',
                 'taster_twitter_handle','winery'], axis=1, inplace = True)
df['price'].fillna(value=df['price'].mean(), inplace=True)
df.price = df.price.round()
df.head()

,country,description,points,price,title,variety
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,35.0,Nicosia 2013 Vulkà Bianco (Etna),White Blend
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir


In [ ]:
df['price'].describe()

count    129971.000000
mean         35.338237
std          39.577174
min           4.000000
25%          18.000000
50%          28.000000
75%          40.000000
max        3300.000000
Name: price, dtype: float64

In [ ]:
df['points'].describe()

count    129971.000000
mean         88.447138
std           3.039730
min          80.000000
25%          86.000000
50%          88.000000
75%          91.000000
max         100.000000
Name: points, dtype: float64

In [ ]:
df['points'] = ["medium rating" if x<87 else "good rating" if 87<=x<=88 else "high rating" if 89<=x<=91 else "top rating" for x in df['points']]
df['price'] = ["low price" if x<19 else "medium price" if 19<=x<=28 else "high price" if 29<=x<=40 else "top price" for x in df['price']]

In [ ]:
df.head()

,country,description,points,price,title,variety
0,Italy,"Aromas include tropical fruit, broom, brimston...",good rating,high price,Nicosia 2013 Vulkà Bianco (Etna),White Blend
1,Portugal,"This is ripe and fruity, a wine that is smooth...",good rating,low price,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red
2,US,"Tart and snappy, the flavors of lime flesh and...",good rating,low price,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris
3,US,"Pineapple rind, lemon pith and orange blossom ...",good rating,low price,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling
4,US,"Much like the regular bottling from 2012, this...",good rating,top price,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir


In [ ]:
df.shape

(129971, 6)

In [ ]:
df.isna().sum()

country        63
description     0
points          0
price           0
title           0
variety         1
dtype: int64

In [ ]:
df.dropna(inplace=True)

### TEXT MINING PRE PROCESSING

In [ ]:
#metadata non considerando le colonne points e price, ma considerando solo country, description e variety
df["metadata"] = df.apply(lambda x : "".join(x['points']) + " " + "" .join(x["price"]) + " " + "".join(x["variety"])+ " " + "".join(x["description"])+ " " + "".join(x["country"]), axis = 1)

In [ ]:
df.head()

,country,description,points,price,title,variety,metadata
0,Italy,"Aromas include tropical fruit, broom, brimston...",good rating,high price,Nicosia 2013 Vulkà Bianco (Etna),White Blend,good rating high price White Blend Aromas incl...
1,Portugal,"This is ripe and fruity, a wine that is smooth...",good rating,low price,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,good rating low price Portuguese Red This is r...
2,US,"Tart and snappy, the flavors of lime flesh and...",good rating,low price,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,good rating low price Pinot Gris Tart and snap...
3,US,"Pineapple rind, lemon pith and orange blossom ...",good rating,low price,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,"good rating low price Riesling Pineapple rind,..."
4,US,"Much like the regular bottling from 2012, this...",good rating,top price,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,good rating top price Pinot Noir Much like the...


In [ ]:
df.drop(columns=['country','points','price','variety'], axis=1, inplace = True)
df.head()

,description,title,metadata
0,"Aromas include tropical fruit, broom, brimston...",Nicosia 2013 Vulkà Bianco (Etna),good rating high price White Blend Aromas incl...
1,"This is ripe and fruity, a wine that is smooth...",Quinta dos Avidagos 2011 Avidagos Red (Douro),good rating low price Portuguese Red This is r...
2,"Tart and snappy, the flavors of lime flesh and...",Rainstorm 2013 Pinot Gris (Willamette Valley),good rating low price Pinot Gris Tart and snap...
3,"Pineapple rind, lemon pith and orange blossom ...",St. Julian 2013 Reserve Late Harvest Riesling ...,"good rating low price Riesling Pineapple rind,..."
4,"Much like the regular bottling from 2012, this...",Sweet Cheeks 2012 Vintner's Reserve Wild Child...,good rating top price Pinot Noir Much like the...


**Pre-processing (stopwords, numeri, lower, puntegg e speciali)**

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() #porta la parola alla sua radice linguistica


raw_1 = df['description']
raw_2 = df['title']
raw_3 = df['metadata'].str.lower().str.replace('[\W\s\d]',' ', regex=True) # W=punteggiatura, s=space, d=numeri
raw_3 = raw_3.str.split() #creiamo una lista con ogni parola della review. Le righe del df['Review'] adesso sono liste di parole  (una lista per ogni riga)
raw_3 = raw_3.apply(lambda x: [lemmatizer.lemmatize(sent) for sent in x])

df_postpro = pd.DataFrame((zip(raw_1,raw_2, raw_3)),
               columns =['description','title', 'metadata'])
  
df_postpro.head()

,description,title,metadata
0,"Aromas include tropical fruit, broom, brimston...",Nicosia 2013 Vulkà Bianco (Etna),"[good, rating, high, price, white, blend, arom..."
1,"This is ripe and fruity, a wine that is smooth...",Quinta dos Avidagos 2011 Avidagos Red (Douro),"[good, rating, low, price, portuguese, red, th..."
2,"Tart and snappy, the flavors of lime flesh and...",Rainstorm 2013 Pinot Gris (Willamette Valley),"[good, rating, low, price, pinot, gris, tart, ..."
3,"Pineapple rind, lemon pith and orange blossom ...",St. Julian 2013 Reserve Late Harvest Riesling ...,"[good, rating, low, price, riesling, pineapple..."
4,"Much like the regular bottling from 2012, this...",Sweet Cheeks 2012 Vintner's Reserve Wild Child...,"[good, rating, top, price, pinot, noir, much, ..."


In [ ]:
df_postpro.to_csv('/content/drive/MyDrive/Web Data Analytics/group_project/df_postpro.csv', index=False)

**READ DATASET**

In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [5]:
df_postpro = pd.read_csv("/content/drive/MyDrive/Web Data Analytics/group_project/df_postpro.csv")
df_postpro = df_postpro.head(20000)
df_postpro.head()

,description,title,metadata
0,"Aromas include tropical fruit, broom, brimston...",Nicosia 2013 Vulkà Bianco (Etna),"['good', 'rating', 'high', 'price', 'white', '..."
1,"This is ripe and fruity, a wine that is smooth...",Quinta dos Avidagos 2011 Avidagos Red (Douro),"['good', 'rating', 'low', 'price', 'portuguese..."
2,"Tart and snappy, the flavors of lime flesh and...",Rainstorm 2013 Pinot Gris (Willamette Valley),"['good', 'rating', 'low', 'price', 'pinot', 'g..."
3,"Pineapple rind, lemon pith and orange blossom ...",St. Julian 2013 Reserve Late Harvest Riesling ...,"['good', 'rating', 'low', 'price', 'riesling',..."
4,"Much like the regular bottling from 2012, this...",Sweet Cheeks 2012 Vintner's Reserve Wild Child...,"['good', 'rating', 'top', 'price', 'pinot', 'n..."


In [6]:
import ast

df_postpro['metadata'] =[ast.literal_eval(x) for x in df_postpro['metadata'] ]
meta_list = df_postpro['metadata'] 

### TEXT MINING WITH METADATA - COUNT VECTORIZER

In [7]:
!pip install gensim==3.8.3

     |████████████████████████████████| 24.2 MB 1.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [10]:
from gensim.models.phrases import Phrases
from nltk.corpus import stopwords
connectors = stopwords.words('english')

bigram = Phrases(meta_list, min_count=3, threshold=0.1, common_terms=connectors)

bigrams = [bigram[item] for item in meta_list]
ngrams = [bigram[item] for item in bigrams]
print(bigrams[3:10])
print(ngrams[3:10])

[['good_rating', 'low_price', 'riesling', 'pineapple', 'rind', 'lemon_pith', 'and', 'orange_blossom', 'start', 'off', 'the', 'aroma_the_palate', 'is', 'a', 'bit', 'more', 'opulent', 'with', 'note_of_honey', 'drizzled', 'guava', 'and', 'mango', 'giving_way', 'to', 'a', 'slightly_astringent', 'semidry', 'finish_u'], ['good_rating', 'top_price', 'pinot_noir', 'much_like', 'the', 'regular_bottling', 'from', 'this', 'come_across', 'a', 'rather', 'rough', 'and', 'tannic', 'with', 'rustic', 'earthy_herbal', 'characteristic', 'nonetheless', 'if', 'you', 'think', 'of', 'it', 'a', 'a', 'pleasantly', 'unfussy', 'country_wine', 'it', 's', 'a', 'good_companion', 'to', 'a', 'hearty', 'winter', 'stew_u'], ['good_rating', 'low_price', 'tempranillo_merlot', 'blackberry_and_raspberry', 'aroma_show', 'a', 'typical', 'navarran', 'whiff', 'of', 'green_herb', 'and', 'in', 'this', 'case', 'horseradish', 'in', 'the', 'mouth', 'this', 'is', 'fairly_full', 'bodied', 'with', 'tomatoey_acidity', 'spicy_herbal', '

In [11]:
stop = stopwords.words('english')
stop.extend(['good', 'bad', 'dont', 'many', 'love', 'excellent', 'would', 'perfect', 'even', 'great','nice'])

metadata = []
for row in ngrams:
    metadata.append(' '.join([item for item in row if item not in stop]))
# train_sentences = [' '.join(item) for item in ngrams]
metadata[5]

'good_rating low_price tempranillo_merlot blackberry_and_raspberry aroma_show typical navarran whiff green_herb case horseradish mouth fairly_full bodied tomatoey_acidity spicy_herbal flavor complement dark_plum fruit finish_is_fresh grabby spain'

In [12]:
raw_2 = df_postpro['title']
raw_4 = df_postpro['description']

df_we_meta = pd.DataFrame((zip(metadata,raw_2, raw_4)),
               columns =['metadata','title', 'description'])
pd.set_option('display.max_colwidth', None)
df_we_meta.head()

,metadata,title,description
0,good_rating high_price white_blend aroma_include tropical_fruit broom brimstone dried_herb palate overly expressive_offering unripened apple_citrus dried_sage alongside_brisk acidity_italy,Nicosia 2013 Vulkà Bianco (Etna),"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity."
1,good_rating low_price portuguese_red ripe_and_fruity wine smooth still_structured firm_tannin filled juicy_red berry_fruit freshened acidity_it_s_already drinkable although certainly_be_better portugal,Quinta dos Avidagos 2011 Avidagos Red (Douro),"This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's already drinkable, although it will certainly be better from 2016."
2,good_rating low_price pinot_gris tart snappy flavor_of_lime flesh_and_rind dominate green pineapple poke crisp_acidity underscoring flavor_the_wine wa stainless_steel fermented u,Rainstorm 2013 Pinot Gris (Willamette Valley),"Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented."
3,good_rating low_price riesling pineapple rind lemon_pith orange_blossom start aroma_the_palate bit opulent note_of_honey drizzled guava mango giving_way slightly_astringent semidry finish_u,St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore),"Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish."
4,good_rating top_price pinot_noir much_like regular_bottling come_across rather rough tannic rustic earthy_herbal characteristic nonetheless think pleasantly unfussy country_wine good_companion hearty winter stew_u,Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley),"Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew."


In [ ]:
df_we_meta.to_csv('/content/drive/MyDrive/Web Data Analytics/group_project/df_we_meta.csv', index=None)

**Count vectorizer**

In [ ]:
count_vec = CountVectorizer()
count_vec_matrix = count_vec.fit_transform(metadata)
cosine_sim_matrix = cosine_similarity(count_vec_matrix, count_vec_matrix)

In [ ]:
print(cosine_sim_matrix)

[[1.         0.06063391 0.06063391 ... 0.         0.         0.        ]
 [0.06063391 1.         0.11764706 ... 0.04950738 0.         0.05882353]
 [0.06063391 0.11764706 1.         ... 0.04950738 0.         0.05882353]
 ...
 [0.         0.04950738 0.04950738 ... 1.         0.14048787 0.14852213]
 [0.         0.         0.         ... 0.14048787 1.         0.05564149]
 [0.         0.05882353 0.05882353 ... 0.14852213 0.05564149 1.        ]]


In [ ]:
#title index mapping
mapping = pd.Series(df_postpro["title"])

In [ ]:
def recommend_meta(title, cosine_sim = cosine_sim_matrix):
    recommended_wine = []
    recommended_metadata = []
    cosine_simz = []
    
    idx = mapping[mapping == title].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_wine.append(df_we_meta['title'][i])
        recommended_metadata.append(df_we_meta['metadata'][i])
        cosine_simz.append(score_series[i])

    score_meta = pd.DataFrame(list(zip(recommended_wine,recommended_metadata, cosine_simz,)),
               columns =['title','metadata', 'val'])    
    return score_meta

In [ ]:
recommend_meta('Firesteed 2007 Pinot Noir (Willamette Valley)')

,title,metadata,val
0,Roux Père et Fils 2006 Beauregard Premier Cru ...,good_rating high_price pinot_noir firm_structu...,0.304290
1,Jaffelin 2014 Chorey-lès-Beaune,good_rating high_price pinot_noir generous_win...,0.284268
2,Anam Cara 2010 Nicholas Estate Pinot Noir (Che...,good_rating high_price pinot_noir pleasant lig...,0.251976
3,Gauthier 2009 Rockin' H Ranch Vineyards Pinot ...,good_rating high_price pinot_noir big full_bod...,0.245737
4,Björnson 2014 Pinot Noir (Eola-Amity Hills),good_rating high_price pinot_noir estate_grown...,0.240563
5,Storm 2009 Pinot Noir (Santa Maria Valley),good_rating high_price pinot_noir worthy follo...,0.235702
6,Apriori 2014 Hicks Family Vineyard Pinot Noir ...,good_rating top_price pinot_noir cola_black ch...,0.228665
7,Renteria 2009 Knittel Vineyard Pinot Noir (Car...,good_rating high_price pinot_noir exotic spicy...,0.228665
8,Sebastiani 2009 Pinot Noir (Russian River Valley),good_rating high_price pinot_noir show_a_lovel...,0.228665
9,Peacock Cellars 2011 Reserve Pinot Noir (Centr...,medium_rating high_price pinot_noir strong_fla...,0.228665


### TEXT MINING WITH METADATA - WORD EMBEDDING

In [ ]:
import pandas as pd
import ast
from sklearn.metrics.pairwise import cosine_similarity

df_we_meta = pd.read_csv("/content/drive/MyDrive/Web Data Analytics/group_project/df_we_meta.csv")

metadata = df_we_meta['metadata']
df_we_meta.head()
metadata.to_list()

['good_rating high_price white_blend aroma_include tropical_fruit broom brimstone dried_herb palate overly expressive_offering unripened apple_citrus dried_sage alongside_brisk acidity_italy',
 'good_rating low_price portuguese_red ripe_and_fruity wine smooth still_structured firm_tannin filled juicy_red berry_fruit freshened acidity_it_s_already drinkable although certainly_be_better portugal',
 'good_rating low_price pinot_gris tart snappy flavor_of_lime flesh_and_rind dominate green pineapple poke crisp_acidity underscoring flavor_the_wine wa stainless_steel fermented u',
 'good_rating low_price riesling pineapple rind lemon_pith orange_blossom start aroma_the_palate bit opulent note_of_honey drizzled guava mango giving_way slightly_astringent semidry finish_u',
 'good_rating top_price pinot_noir much_like regular_bottling come_across rather rough tannic rustic earthy_herbal characteristic nonetheless think pleasantly unfussy country_wine good_companion hearty winter stew_u',
 'good

In [ ]:
# Import gensim modules.
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Tagging documents. Each sentences(set of words) are mapped unique index.
# Tagged documents are input for doc2vec model. 
tagged_documents = []
for i, doc in enumerate(metadata):
    tagged = TaggedDocument(doc, [i])
    tagged_documents.append(tagged)

tagged_documents

[TaggedDocument(words='good_rating high_price white_blend aroma_include tropical_fruit broom brimstone dried_herb palate overly expressive_offering unripened apple_citrus dried_sage alongside_brisk acidity_italy', tags=[0]),
 TaggedDocument(words='good_rating low_price portuguese_red ripe_and_fruity wine smooth still_structured firm_tannin filled juicy_red berry_fruit freshened acidity_it_s_already drinkable although certainly_be_better portugal', tags=[1]),
 TaggedDocument(words='good_rating low_price pinot_gris tart snappy flavor_of_lime flesh_and_rind dominate green pineapple poke crisp_acidity underscoring flavor_the_wine wa stainless_steel fermented u', tags=[2]),
 TaggedDocument(words='good_rating low_price riesling pineapple rind lemon_pith orange_blossom start aroma_the_palate bit opulent note_of_honey drizzled guava mango giving_way slightly_astringent semidry finish_u', tags=[3]),
 TaggedDocument(words='good_rating top_price pinot_noir much_like regular_bottling come_across r

In [ ]:
max_epochs = 10
vec_size = 20
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                workers=4)
  
model.build_vocab(tagged_documents)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_documents,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("/content/drive/MyDrive/Web Data Analytics/group_project/d2v.model")
print("Model Saved")

iteration 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
Model Saved


In [ ]:
model= Doc2Vec.load("/content/drive/MyDrive/Web Data Analytics/group_project/models/d2v.model")

In [ ]:
print(model.docvecs[2])

[ 0.93275785  0.02159763 -0.4260801  -0.09523636 -0.4366938  -0.10682587
  0.23218364  1.15199    -0.42493227 -0.3456654   0.1596065   0.11733252
  0.43837067  0.2859359  -0.6560154   0.4239519  -0.3798735  -0.41568932
  0.38020664  0.6425339 ]


In [ ]:
embedding = []
for i in range(0,len(tagged_documents)):
    tagged = model.docvecs[i]
    embedding.append(tagged)

cosine_sim_matrix_emb = cosine_similarity(embedding, embedding)

In [ ]:
mapping = pd.Series(df_we_meta["title"])

In [ ]:
def recommend_meta_we(title, cosine_sim = cosine_sim_matrix_emb):
    recommended_wine = []
    recommended_description = []
    recommended_metadata = []
    cosine_simz = []
    
    idx = mapping[mapping == title].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_wine.append(df_we_meta['title'][i])
        recommended_description.append(df_we_meta['description'][i])
        recommended_metadata.append(df_we_meta['metadata'][i])
        cosine_simz.append(score_series[i])

    score_meta = pd.DataFrame(list(zip(recommended_wine, metadata, cosine_simz)),
               columns =['title','metadata', 'val'])    
    return score_meta

In [ ]:
recommend_meta_we('Firesteed 2007 Pinot Noir (Willamette Valley)')

,title,metadata,val
0,Zenato 2001 Vigneto Massoni White (Naoussa),good_rating high_price white_blend aroma_inclu...,0.922220
1,Château la Varière 2010 La Chevalerie Cabernet...,good_rating low_price portuguese_red ripe_and_...,0.915340
2,Tsillan 2013 Estate Viognier (Lake Chelan),good_rating low_price pinot_gris tart snappy f...,0.911692
3,Massimo Daldin 2015 Vernaccia di San Gimignano,good_rating low_price riesling pineapple rind ...,0.897361
4,Two Vines 2013 Chardonnay (Washington),good_rating top_price pinot_noir much_like reg...,0.896696
5,Caves Aliança 2007 Quinta das Baceladas Red (B...,good_rating low_price tempranillo_merlot black...,0.895373
6,Domaine Specht 2008 Pinot Blanc (Alsace),good_rating low_price frappato bright_informal...,0.893570
7,Hibou 2014 Soulajule Pinot Noir (Marin County),good_rating medium_price gewürztraminer dry re...,0.890949
8,Beresini Vineyards 2013 Black Dog Ranch Pinot ...,good_rating low_price gewürztraminer savory_dr...,0.887850
9,Louis Jadot 2009 Mâcon-Villages,good_rating medium_price pinot_gris ha_great d...,0.887632
